In [ ]:
#import sys
#!{sys.executable} -m pip install ortools 

import pandas as pd
import numpy as np
import time
import random
from ortools.linear_solver import pywraplp

In [ ]:
InputData = pd.read_excel("Riverside.xlsx", sep="\t")
InputData

In [ ]:
# Determine the start time
StartTime = time.process_time()

# Define our Linear Program
Solver = pywraplp.Solver('Solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Define the Preference Coefficient P[i,j], for Residence i working in Week j
n=18
m=24

P = np.zeros(shape=(n, m), dtype=int)
blocks = ["B1W1","B1W2","B1W3","B2W1","B2W2","B2W3","B3W1","B3W2","B3W3","B4W1","B4W2","B4W3",
"B5W1","B5W2","B5W3","B6W1","B6W2","B6W3","B7W1","B7W2","B7W3","B8W1","B8W2","B8W3"]

for j in range(m):
    for i in range(n):
        P[i,j] = InputData[blocks[j]][i]
        
# Define the binary variable X[i,j], which will equal 1 if Residence i is assigned to Week j
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = Solver.IntVar(0, 1, 'X[%d, %d]' % (i,j))
        
# Set up our Happiness Function, which maximizes the total number of Happiness Points
HappinessFunction = Solver.Sum(P[i,j]*X[i,j] for i in range(n) for j in range(m))
Solver.Maximize(HappinessFunction)

# Include our first constraint: Each residence must host two parties in the year
for i in range(n):
    Solver.Add(Solver.Sum([X[i,j] for j in range(12)]) == 1)
    Solver.Add(Solver.Sum([X[i,j] for j in range(m)]) == 2)
                       
# Include our second constraint: Each week must be covered by at least one residence and at most two
for j in range(m):
    Solver.Add(Solver.Sum([X[i,j] for i in range(n)]) >= 1)
    Solver.Add(Solver.Sum([X[i,j] for i in range(n)]) <= 2)

# Solve the Integer Linear program
Output = Solver.Solve()
TotalPoints = round(Solver.Objective().Value())

# Determine the total time of running the program.
TotalTime = round(time.process_time() - StartTime, 4)

# Output one of the possible optimal solutions.
print("Python returns a solution with", TotalPoints, "Total Happiness Points in", TotalTime, "seconds")
pd.options.mode.chained_assignment = None
OutputData = InputData.copy()
for i in range(n):
    for j in range(m):
        if X[i,j].solution_value()==0:
            OutputData[blocks[j]][i] = ""
OutputData 

In [ ]:
for i in range(n):
    for j in range(m):
        if X[i,j].solution_value()==1:
            print("Residence", i+1, "will host in", blocks[j], "with score", P[i,j])

In [ ]:
for j in range(m):
    for i in range(n):
        if X[i,j].solution_value()==1:
            print(blocks[j], "will be hosted by Residence", i+1, "with score", P[i,j])